In [1]:
import tensorflow as tf 
from tensorflow import keras

2024-12-13 17:08:24.228524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734080904.240569   16614 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734080904.244015   16614 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 17:08:24.257125: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.18.0'

In [4]:
# load original model
model = keras.models.load_model('model_2024_hairstyle.keras')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('hairstyle-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp1ngoxmj6/assets


INFO:tensorflow:Assets written to: /tmp/tmp1ngoxmj6/assets


Saved artifact at '/tmp/tmp1ngoxmj6'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  125009495370816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125009495663792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125009495798032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125009495796800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125009495790816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125009456074944: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1734080915.678970   16614 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1734080915.678981   16614 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2024-12-13 17:08:35.679117: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp1ngoxmj6
2024-12-13 17:08:35.679380: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-12-13 17:08:35.679390: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp1ngoxmj6
2024-12-13 17:08:35.681879: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-12-13 17:08:35.750503: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp1ngoxmj6
2024-12-13 17:08:35.756000: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 76884 microseconds.


In [5]:
!ls -lh

total 230M
-rw-rw-r-- 1 kabs kabs  77M Dec 13 17:08 hairstyle-model.tflite
-rw-rw-r-- 1 kabs kabs 154M Dec 13 16:38 model_2024_hairstyle.keras
-rw-rw-r-- 1 kabs kabs 5.6K Dec 13 17:01 notebook.ipynb


In [7]:
!pip3 install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 2.3 MB/s eta 0:00:002.6 MB/s eta 0:00:01


In [1]:
# tflite_runtime and keras-image-helper
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

interpreter = tflite.Interpreter(model_path='hairstyle-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:
output_index

13

In [3]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [4]:
image = download_image('https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg')

prepared_img = prepare_image(image, target_size=(200, 200))

In [5]:
import numpy as np

def prepare_input(x):
    return x / 255.0

x = np.array(prepared_img, dtype='float32')
X = np.array([x])

X = prepare_input(X)

X[0][0][0][0]

0.23921569

In [6]:
# initialize input
interpreter.set_tensor(input_index, X)

# invoke the interpreter
interpreter.invoke()

# fetch output
preds = interpreter.get_tensor(output_index)

In [7]:
preds

array([[0.89377415]], dtype=float32)